In [1]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.llms import HuggingFaceHub
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS, Chroma
from langchain.document_loaders import PyMuPDFLoader
from langchain.chains import RetrievalQA
import os
from dotenv import load_dotenv

In [2]:
load_dotenv()

True

In [3]:
huggingface_token = os.getenv("HUGGINGFACEHUB_API_TOKEN")
os.environ["HUGGINGFACEHUB_API_TOKEN"] = huggingface_token

In [4]:
from langchain_huggingface import HuggingFaceEndpoint

In [5]:
llama_model = HuggingFaceEndpoint(
    repo_id="mistralai/Mistral-7B-Instruct-v0.3",
    max_length=256,
    temperature=0.7,
    token=huggingface_token,
    task="text-generation"
)

WARNING! max_length is not default parameter.
                    max_length was transferred to model_kwargs.
                    Please make sure that max_length is what you intended.
WARNING! token is not default parameter.
                    token was transferred to model_kwargs.
                    Please make sure that token is what you intended.
/Users/saumyavarshney/Documents/MSDA/SEMESTER - FALL'24/Data 255 - Deep Learning/Labs/LAB - 2/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /Users/saumyavarshney/.cache/huggingface/token
Login successful


In [6]:
# Checking the model loaded successfully
llama_model.invoke("What is machine learning")

'? Machine learning is a subset of artificial intelligence that allows computer systems to automatically learn and improve from experience without being explicitly programmed. It focuses on the development of algorithms and statistical models that computers use to perform tasks, such as classification, regression, clustering, and dimensionality reduction. Machine learning algorithms use data to learn patterns and relationships, and then use these patterns to make predictions or decisions about new data.\n\nThere are three main types of machine learning: supervised learning, unsupervised learning, and reinforcement learning. Supervised learning involves training a model using labeled data, where the correct output is known for each example. The model learns to predict the output for new, unseen examples based on the patterns it has learned from the labeled data. Unsupervised learning involves training a model on unlabeled data, where the goal is to find patterns or structure in the data

In [7]:
pdf_file = "Lab2_Part3_Data/Meta-Reports-Fourth-Quarter-and-Full-Year-2023-Results-Initiates-Quarterly-Dividend-2024.pdf"
loader = PyMuPDFLoader(pdf_file)
documents = loader.load()

In [8]:
# Splitting Document into Chunks
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
texts = [chunk for doc in documents for chunk in splitter.split_text(doc.page_content)]

In [9]:
#Embedding Model
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

/var/folders/dd/xt_pnsf92hd2vwfkqlyq5zfc0000gn/T/ipykernel_96339/4270316567.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


In [10]:
vector_store = Chroma.from_texts(texts, embedding_model, persist_directory="chroma_db")
vector_store.embeddings


HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [11]:
# Retrieval-Augmented Generation (RAG) with Langchain
qa_chain = RetrievalQA.from_chain_type(
    llm=llama_model,
    chain_type="stuff",
    retriever=vector_store.as_retriever()
)

In [12]:
# Q and A
query = "What is the main point of the document?" 
response = qa_chain.run(query)
print("Response:", response)

Response:  The main point of the document is to outline potential risks and uncertainties that could affect the results of Facebook, including but not limited to government inquiries, privacy concerns, risks associated with acquisitions, security breaches, market conditions, geopolitical events, reliance on advertising revenue, competition, and government actions that could restrict access to their products.


/var/folders/dd/xt_pnsf92hd2vwfkqlyq5zfc0000gn/T/ipykernel_96339/2317917202.py:3: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = qa_chain.run(query)


In [13]:
query1 = "What were the key financial highlights this quarter(revenue, gross margin, operating expenses, operating margin, net income, and EPS. )?" 
response = qa_chain.run(query1)
print("Response:", response)

Response: 


In [14]:
query2 = "What is the report quarter, and when did it end?" 
response = qa_chain.run(query2)
print("Response:", response)

Response:  The report quarter is the fourth quarter of 2023, and it ended on December 31, 2023.
